In [2]:
# import needed data
import pandas as pd
import numpy as np
import subprocess
import os

In [3]:
def blastn_dic(path_input):
    os.system("makeblastdb -in " + path_input + " -dbtype nucl -parse_seqids")
    print("\nBlast Dictionary created in", path_input)
    
def blastn_blaster(query, db, perc_indentity):
    cmd = "blastn -word_size 11 " \
    + " -query " + query \
    + " -db "  + db \
    + " -perc_identity " + str(perc_indentity) \
    + " -outfmt '10 qseqid sseqid pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp sstrand sseq'"
    data = subprocess.check_output(cmd, shell=True, universal_newlines=True)
    data = pd.DataFrame([x.split(",") for x in data.split("\n") if x])
    data_col = data.copy()

    data_col.columns = ["qseqid", "sseqid", "pident", "length", "qlen", "slen", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qcovhsp", "sstrand", "sseq"]

    return data, data_col

In [4]:
blastn_dic("./dict/mine_chr1.fasta")



Building a new DB, current time: 04/30/2024 09:53:46
New DB name:   /home/rfpacheco/Desktop/Projects/Testing_Leishmania_project/6.subfamilies_test_1/dict/mine_chr1.fasta
New DB title:  ./dict/mine_chr1.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/rfpacheco/Desktop/Projects/Testing_Leishmania_project/6.subfamilies_test_1/dict/mine_chr1.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 12 sequences in 0.118952 seconds.



Blast Dictionary created in ./dict/mine_chr1.fasta


In [5]:
data, data_col = blastn_blaster("./dict/mine_chr1.fasta",
                    "./dict/mine_chr1.fasta",
                    85)
print(type(data), len(data), sep="\n")
print(type(data_col), len(data_col), sep="\n")

<class 'pandas.core.frame.DataFrame'>
115
<class 'pandas.core.frame.DataFrame'>
115


In [6]:
# Filter length < 100
data_col_2 = data_col[data_col["length"].astype(int) > 100]  # Need the column to be int, it's str by default

In [7]:
data_col_3 = data_col_2.query("qseqid != sseqid")  # Remove self matches

In [8]:
data_col_3[data_col_3["qseqid"] == "Seq_2"]

,qseqid,sseqid,pident,length,qlen,slen,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qcovhsp,sstrand,sseq
4,Seq_2,Seq_5,99.534,644,699,888,3,0,23,666,644,1,0.0,1173,92,minus,GGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCCCCCCC...
6,Seq_2,Seq_3,99.192,619,699,845,3,2,7,624,618,1,0.0,1114,88,minus,GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGG...
8,Seq_2,Seq_6,98.441,449,699,868,1,5,30,477,265,708,0.0,785,64,plus,ACCT-CA-GCGTGGCATCCCAGGGTCCAGCG--CCCCCCCTCCACC...
10,Seq_2,Seq_11,86.207,174,699,424,21,3,189,362,171,341,6.10e-50,185,25,plus,GCGCGAGTTAGGGCTACGGACGTCAGCGGCCATGTCGTGCATGGCG...
12,Seq_2,Seq_9,86.207,174,699,609,21,3,189,362,215,385,6.10e-50,185,25,plus,GCGCGAGTTAGGGCTACGGACGTCAGCGGCCATGTCGTGCATGGCG...
14,Seq_2,Seq_8,85.632,174,699,803,22,3,189,362,253,83,2.84e-48,180,25,minus,GCGCGAGTTAGGGCTACGGACGTCAGCGGCCATGTCGTGCATGGCG...


In [9]:
data_col_3[data_col_3["qseqid"] == "Seq_2"].loc[4, ["sseq"]].values[0]

'GGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCCCCCCCCTCCACCCCCGCTCTCTCTGTGTACGGAAGCCCGGCAGCCCCCTCACCCTCTATCCCTGCCAACGCCGAACCACTTCTGGTGCTGACAGGGCTGGGCCCTTACGGCGTTGGGCGAGGTCGGCGCGACGTAGCGCCACGGATGCCGGCGGCCATGTCGTGCATGGCGCTGCGTCCGAGCCACCCGCGACAGTGAGCACAGGCTTGCACGGTCCATGCGATGGGCGGAGCGTGTCAGCGCGGCTCGAGCGTGTCGCAGCCGGCCCCTCACACTGGCCCACGGCGAGGGGTGCGGGGGCCTGAGTGTCACCGCGACGGGGAGACGCACCCAGGGAGGAGGTGGGGGGAGTGGGGACCGGCATGATGGAGGGGCGGCTGTGTGGCGATGTGTGTGTATGTGGGTGTCGTCGCGTTTGAGGCAGGAGCCGTGCTGCTATGACCGAGCCGGCGCACTGCTGCAGCGCGCGTGTGTCTTGCGCTGCTTCGCACCAGGCGATGAGAGTGGGGTGGGGTGCCTGCAGCACTCGGCGGCGGGGGGGTGCAGAGCGGCCTCCACCTCGCAGTGTGCGGCAGCGAAAACGGACACGCGGAAGACCAAACA'

In [10]:
data_col_2.head()

,qseqid,sseqid,pident,length,qlen,slen,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qcovhsp,sstrand,sseq
0,Seq_1,Seq_1,100.000,173,173,173,0,0,1,173,1,173,3.76e-91,320,100,plus,ACACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCA...
1,Seq_2,Seq_2,100.000,699,699,699,0,0,1,699,1,699,0.0,1291,100,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
4,Seq_2,Seq_5,99.534,644,699,888,3,0,23,666,644,1,0.0,1173,92,minus,GGCACGCACCTCCATGCGTGGCATCCCAGGGTCCAGCGCCCCCCCC...
6,Seq_2,Seq_3,99.192,619,699,845,3,2,7,624,618,1,0.0,1114,88,minus,GGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATCCCAGG...
8,Seq_2,Seq_6,98.441,449,699,868,1,5,30,477,265,708,0.0,785,64,plus,ACCT-CA-GCGTGGCATCCCAGGGTCCAGCG--CCCCCCCTCCACC...


In [11]:
# Let'screate a dictionary with the sequences
dict = {}
for seq in data_col_2["qseqid"].unique():
    values = data_col_2[data_col_2["qseqid"] == seq].loc[:, ["sseqid"]].values.flatten().tolist()
    values = sorted(values)
    dict[seq] = values

dict = {key: sorted(value, key=lambda x: int(x.split('_')[1])) for key, value in dict.items()}   # Sort the values by the number of the sequence


In [13]:
dict

{'Seq_1': ['Seq_1'],
 'Seq_2': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_3': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_4': ['Seq_4', 'Seq_7'],
 'Seq_5': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_6': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_7': ['Seq_4', 'Seq_7'],
 'Seq_8': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_9': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_10': ['Seq_10'],
 'Seq_11': ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 'Seq_12': ['Seq_12']}

In [16]:
dict.values()

dict_values([['Seq_1'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_4', 'Seq_7'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_4', 'Seq_7'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_10'], ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'], ['Seq_12']])

In [17]:
dict.keys()

dict_keys(['Seq_1', 'Seq_2', 'Seq_3', 'Seq_4', 'Seq_5', 'Seq_6', 'Seq_7', 'Seq_8', 'Seq_9', 'Seq_10', 'Seq_11', 'Seq_12'])

In [22]:
value_count = {}
for value in dict.values(): 
    print(value)
    print(type(value))

['Seq_1']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_4', 'Seq_7']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_4', 'Seq_7']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_10']
<class 'list'>
['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11']
<class 'list'>
['Seq_12']
<class 'list'>


In [30]:
dataset = []
for key, value in dict.items():
    if value in dataset:
        continue  # If it's in the dataset, it doesn't do anything.
    else:
        dataset.append(value)  # If it's not in the dataset, it appends it.
dataset

[['Seq_1'],
 ['Seq_2', 'Seq_3', 'Seq_5', 'Seq_6', 'Seq_8', 'Seq_9', 'Seq_11'],
 ['Seq_4', 'Seq_7'],
 ['Seq_10'],
 ['Seq_12']]